### Gerekli Kütüphaneler

In [1]:
import psycopg2
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import re

### HTML Sayfalarını Çekme - Test

In [8]:
conn = psycopg2.connect(
    dbname="websites",
    user="postgres",
    password="1234567",
    host="localhost",
    port=5432
)

query = "SELECT * FROM pages LIMIT 5;"  # Başlangıç için sınırlı al
df = pd.read_sql(query, conn)
df[:5]

/var/folders/gs/yjn9xz_d6qn151z0ncgbc9gm0000gn/T/ipykernel_98284/3891044262.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,id,url,title,html,created_at
0,32720,https://crimethinc.com/feed/ro.json,,"{\n ""version"": ""https://jsonfeed.org/version/...",2025-05-16 20:17:02.954342+00:00
1,32743,https://crimethinc.com/feed/tr.json,,"{\n ""version"": ""https://jsonfeed.org/version/...",2025-05-16 20:17:02.954342+00:00
2,32744,https://crimethinc.com/feed/uk,CrimethInc. : UK,"<?xml version=""1.0"" encoding=""UTF-8""?>\n\n<fee...",2025-05-16 20:17:02.954342+00:00
3,32745,https://crimethinc.com/feed/uk.json,,"{\n ""version"": ""https://jsonfeed.org/version/...",2025-05-16 20:17:02.954342+00:00
4,25487,https://www.birgun.net/haber/patron-mangalda-i...,"Patron mangalda, işçiler grevde",<!DOCTYPE html><html lang=tr><head><script>!fu...,2025-05-16 20:09:00.214337+00:00


In [4]:
def extract_text_and_links(html_content, base_url):
    soup = BeautifulSoup(html_content, "html.parser")

    # Metin çıkarımı
    for tag in soup(["script", "style", "noscript"]):
        tag.extract()

    text = soup.get_text(separator=' ', strip=True)

    # Hyperlink çıkarımı
    links = []
    for a_tag in soup.find_all("a", href=True):
        href = urljoin(base_url, a_tag['href'])  # relative → absolute
        if urlparse(href).scheme in ['http', 'https']:
            links.append(href)

    return text, links


In [9]:
processed = []

for _, row in df.iterrows():
    doc_id = row['id']
    url = row['url']
    html = row['html']

    try:
        text, links = extract_text_and_links(html, url)
        processed.append({
            'id': doc_id,
            'url': url,
            'text': text,
            'links': links
        })
    except Exception as e:
        print(f"Hata ({url}): {e}")

df_processed = pd.DataFrame(processed)
df_processed[:5]

/opt/anaconda3/lib/python3.12/html/parser.py:171: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


,id,url,text,links
0,32720,https://crimethinc.com/feed/ro.json,"{\n ""version"": ""https://jsonfeed.org/version/...","[https://crimethinc.com/feed/\""https:/crimethi..."
1,32743,https://crimethinc.com/feed/tr.json,"{\n ""version"": ""https://jsonfeed.org/version/...","[https://crimethinc.com/feed/\""#fn:1\"", https:..."
2,32744,https://crimethinc.com/feed/uk,https://crimethinc.com/feed/uk CrimethInc. : U...,[]
3,32745,https://crimethinc.com/feed/uk.json,"{\n ""version"": ""https://jsonfeed.org/version/...","[https://crimethinc.com/feed/\""#fn:1\"", https:..."
4,25487,https://www.birgun.net/haber/patron-mangalda-i...,"Patron mangalda, işçiler grevde Giriş / Abone ...","[https://abone.birgun.net/, https://play.googl..."


In [19]:
import html.entities  # html modülünü doğru şekilde import ediyoruz

def safe_encode_decode(text):
    """
    Metni güvenli bir şekilde encode ve decode eder
    """
    if not text:
        return ""
    
    try:
        # Önce HTML karakterlerini decode et
        decoded = html.unescape(text)
        
        # UTF-8'e çevir ve hatalı karakterleri atla
        encoded = decoded.encode('utf-8', errors='ignore')
        
        # Tekrar decode et
        return encoded.decode('utf-8')
    except Exception as e:
        print(f"Metin dönüştürme hatası: {e}")
        return ""

In [21]:
import psycopg2
from psycopg2.extras import execute_values
from tqdm import tqdm

def process_in_chunks(batch_size=1000):
    # PostgreSQL bağlantı parametreleri
    db_params = {
        'dbname': 'websites',
        'user': 'postgres',
        'password': '1234567',  # Kendi şifrenizi kullanın
        'host': 'localhost',
        'port': '5432'
    }
    
    # Veritabanı bağlantısı
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    
    try:
        # Toplam kayıt sayısını al
        cursor.execute("SELECT COUNT(*) FROM pages")
        total_records = cursor.fetchone()[0]
        
        # Offset ile parça parça veri çek
        offset = 0
        with tqdm(total=total_records, desc="İşlenen Kayıtlar") as pbar:
            while True:
                # Belirli sayıda kayıt çek
                cursor.execute(f"SELECT id, url, title, html FROM pages LIMIT {batch_size} OFFSET {offset}")
                rows = cursor.fetchall()
                
                if not rows:
                    break
                    
                # Her bir kayıt için işlem yap
                for row in rows:
                    id = row[0]
                    url = row[1]
                    title = row[2]
                    html = safe_encode_decode(row[3])
                    text, links = extract_text_and_links(html, url)
                    cursor.execute("""
                        INSERT INTO pages_cleaned (id, url, text, links)
                        VALUES (%s, %s, %s, %s)
                        ON CONFLICT (id) DO UPDATE
                        SET text = EXCLUDED.text, links = EXCLUDED.links;
                    """, (id, url, text, links))

                    pass
                
                # İlerleme çubuğunu güncelle
                pbar.update(len(rows))
                
                # Offset'i güncelle
                offset += batch_size
                
                # Her batch'te commit yap
                conn.commit()
        
        print(f"Toplam {total_records} kayıt işlendi.")
        
    except Exception as e:
        print(f"Hata oluştu: {e}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

# Kodu çalıştır
process_in_chunks(batch_size=1000)

İşlenen Kayıtlar: 100%|██████████| 98984/98984 [1:59:35<00:00, 13.80it/s]  

Toplam 98984 kayıt işlendi.
